# execute_values 사용 
    - 줄바꿈 천처리 사용 x 그대로 삽입

In [ ]:
import pandas as pd
from psycopg2 import connect
from psycopg2.extras import execute_values

import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()
# PostgreSQL 연결 설정
conn_params = {
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'database': os.getenv('DB_DATABASE'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

# 엑셀 파일을 읽어 데이터프레임으로 변환
excel_file_path = 'C:/Users/sphong/hong/GOODS_BASE_202410301537_2.xlsx'
df = pd.read_excel(excel_file_path, sheet_name=0)

# 칼럼 이름을 소문자로 변환
df.columns = [col.lower() for col in df.columns]

# 테이블 이름 설정
table_name = 'product_code_info'  # 실제 적재할 테이블 이름을 입력하세요

# PostgreSQL 연결
try:
    # 데이터베이스 연결 설정
    conn = connect(
        host=conn_params['host'],
        port=conn_params['port'],
        database=conn_params['database'],
        user=conn_params['user'],
        password=conn_params['password']
    )
    cursor = conn.cursor()
    
    # SQL 구문과 값을 생성
    columns = ', '.join(df.columns)
    sql = f"INSERT INTO {table_name} ({columns}) VALUES %s"
    values = [tuple(row) for row in df.to_numpy()]  # 데이터프레임의 각 행을 튜플로 변환하여 리스트에 저장

    # execute_values를 사용하여 여러 행 삽입
    execute_values(cursor, sql, values)
    
    # 커밋하여 변경 사항 저장
    conn.commit()
    print("DataFrame has been successfully loaded to the PostgreSQL database using execute_values.")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # 커서와 연결 닫기
    if cursor:
        cursor.close()
    if conn:
        conn.close()


DataFrame has been successfully loaded to the PostgreSQL database using execute_values.


    - 줄바꿈 천처리 사용하여 데이터 삽입
    - df = df.applymap(lambda x: x.replace('\n', '').replace('\t', '') if isinstance(x, str) else x)

In [ ]:
import pandas as pd
from psycopg2 import connect
from psycopg2.extras import execute_values

load_dotenv()
# PostgreSQL 연결 설정
conn_params = {
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'database': os.getenv('DB_DATABASE'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

# 엑셀 파일을 읽어 데이터프레임으로 변환
excel_file_path = 'C:/Users/sphong/hong/GOODS_BASE_202410301537_1.xlsx'
df = pd.read_excel(excel_file_path, sheet_name=0)

# 칼럼 이름을 소문자로 변환
df.columns = [col.lower() for col in df.columns]

# 줄바꿈(\n)과 탭(\t) 제거
df = df.applymap(lambda x: x.replace('\n', '').replace('\t', '') if isinstance(x, str) else x)

# 테이블 이름 설정
table_name = 'product_code_info2'  # 실제 적재할 테이블 이름을 입력하세요

# PostgreSQL 연결
try:
    # 데이터베이스 연결 설정
    conn = connect(
        host=conn_params['host'],
        port=conn_params['port'],
        database=conn_params['database'],
        user=conn_params['user'],
        password=conn_params['password']
    )
    cursor = conn.cursor()
    
    # SQL 구문과 값을 생성
    columns = ', '.join(df.columns)
    sql = f"INSERT INTO {table_name} ({columns}) VALUES %s"
    values = [tuple(row) for row in df.to_numpy()]  # 데이터프레임의 각 행을 튜플로 변환하여 리스트에 저장

    # execute_values를 사용하여 여러 행 삽입
    execute_values(cursor, sql, values)
    
    # 커밋하여 변경 사항 저장
    conn.commit()
    print("DataFrame has been successfully loaded to the PostgreSQL database using execute_values.")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # 커서와 연결 닫기
    if cursor:
        cursor.close()
    if conn:
        conn.close()


C:\Users\클루커스\AppData\Local\Temp\ipykernel_12108\2793459269.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('\n', '').replace('\t', '') if isinstance(x, str) else x)


DataFrame has been successfully loaded to the PostgreSQL database using execute_values.


# COPY FROM 사용 및 전처리

In [ ]:
import pandas as pd
import psycopg2
from io import StringIO
import csv

# 파일 경로 설정
excel_file_path = 'C:/Users/sphong/hong/GOODS_BASE_202410301537_2.xlsx'
df = pd.read_excel(excel_file_path, sheet_name=0)

# 줄바꿈(\n)을 '|'로, 탭(\t)은 빈 문자열로 변경
df = df.applymap(lambda x: x.replace('\n', '|').replace('\t', '') if isinstance(x, str) else x)

load_dotenv()
# PostgreSQL 연결 설정
conn_params = {
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'database': os.getenv('DB_DATABASE'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

# PostgreSQL 연결
conn = psycopg2.connect(**conn_params)
cur = conn.cursor()

# DataFrame을 CSV 형식의 문자열로 변환 (quotechar 제거)
output = StringIO()
df.to_csv(output, sep='\t', index=False, header=False, quoting=csv.QUOTE_NONE, escapechar='\\')  # 따옴표 없이 저장
output.seek(0)  # StringIO 버퍼를 처음으로 되돌림

# 테이블에 데이터 적재
try:
    cur.copy_from(output, 'product_code_info4', null='', sep='\t')
    conn.commit()
    print("Data has been successfully inserted into product_code_info4.")
except Exception as e:
    conn.rollback()
    print("Error while inserting data:", e)
finally:
    cur.close()
    conn.close()


C:\Users\클루커스\AppData\Local\Temp\ipykernel_12108\3183981303.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('\n', '|').replace('\t', '') if isinstance(x, str) else x)


Data has been successfully inserted into product_code_info4.
